In [2]:
import pandas as pd, numpy as np
from pathlib import Path
import zarr
import pandas as pd
from collections import Counter

# With NO Background

In [1]:
import pandas as pd

in_csv = "/hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/combined_meta_full_downsampled.csv"
out_csv = "/hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/combined_meta_full_downsampled_no_bonecells.csv"

label_col = "cell_type"
# list of classes to drop
drop_classes = [
    "Bone marrow cancer cells",
    "Osteocytes",
    "Osteoblasts",
    "Osteoclasts",
    "Chondrocytes"
    # add more labels here...
]

# stream in chunks and filter
chunksize = 200_000
first = True
kept = 0
dropped = 0

for chunk in pd.read_csv(in_csv, chunksize=chunksize):
    mask = ~chunk[label_col].isin(drop_classes)
    kept += mask.sum()
    dropped += (~mask).sum()
    chunk = chunk[mask]
    chunk.to_csv(out_csv, mode="w" if first else "a", index=False, header=first)
    first = False

print(f"Done. Wrote {out_csv}")
print(f"Kept rows: {kept:,}")
print(f"Dropped rows: {dropped:,}")

Done. Wrote /hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/combined_meta_full_downsampled_no_bonecells.csv
Kept rows: 2,095,668
Dropped rows: 88,728


In [2]:
import csv, math, random
from collections import Counter, defaultdict

in_csv   = "/hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/combined_meta_full_downsampled_no_bonecells.csv"
out_csv  = "/hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/combined_meta_full_downsampled_no_bonecells_subset20pct.csv"
label_col = "cell_type"
subset_frac = 0.20
seed = 23

random.seed(seed)

def count_classes(path, label_col):
    counts = Counter()
    with open(path, "r", newline="") as f:
        reader = csv.reader(f)
        header = next(reader)
        y_idx = header.index(label_col)
        for row in reader:
            counts[row[y_idx]] += 1
    return counts, header, y_idx

def compute_targets(counts, frac):
    float_counts = {c: counts[c] * frac for c in counts}
    floor_counts = {c: int(math.floor(v)) for c, v in float_counts.items()}
    remainder = int(round(sum(float_counts.values()))) - sum(floor_counts.values())
    # distribute remainder based on largest fractional part
    frac_parts = sorted(
        ((c, float_counts[c] - floor_counts[c]) for c in counts),
        key=lambda x: x[1],
        reverse=True,
    )
    targets = floor_counts.copy()
    for i in range(remainder):
        targets[frac_parts[i][0]] += 1
    return targets

def stream_subset(in_csv, out_csv, header, y_idx, targets):
    remaining_in_class = defaultdict(int)
    with open(in_csv, "r", newline="") as f:
        reader = csv.reader(f)
        _ = next(reader)
        for row in reader:
            remaining_in_class[row[y_idx]] += 1

    rem_targets = targets.copy()

    with open(in_csv, "r", newline="") as f_in, open(out_csv, "w", newline="") as f_out:
        reader = csv.reader(f_in)
        writer = csv.writer(f_out)
        writer.writerow(header)
        _ = next(reader)
        for row in reader:
            c = row[y_idx]
            if rem_targets.get(c, 0) <= 0:
                remaining_in_class[c] -= 1
                continue
            prob = rem_targets[c] / remaining_in_class[c]
            if random.random() <= prob:
                writer.writerow(row)
                rem_targets[c] -= 1
            remaining_in_class[c] -= 1

def summarize(path, label_col):
    counts = Counter()
    total = 0
    with open(path, "r", newline="") as f:
        reader = csv.reader(f)
        header = next(reader)
        y_idx = header.index(label_col)
        for row in reader:
            counts[row[y_idx]] += 1
            total += 1
    dist = {k: v / total for k, v in counts.items()} if total > 0 else {}
    return total, counts, dist

# Run
counts, header, y_idx = count_classes(in_csv, label_col)
targets = compute_targets(counts, subset_frac)
stream_subset(in_csv, out_csv, header, y_idx, targets)
total, counts_sub, dist_sub = summarize(out_csv, label_col)

print("Original total:", sum(counts.values()))
print("Subset total:", total)
print("Example original class counts:", dict(list(counts.items())[:5]))
print("Example subset class counts:", dict(list(counts_sub.items())[:5]))
print(f"\nSaved 20% subset to {out_csv}")


Original total: 2095668
Subset total: 419134
Example original class counts: {'Astrocytes': 100000, 'B cells': 100000, 'Breast cancer cells': 100000, 'Cardiac muscle cells': 58042, 'Colon cancer cells': 100000}
Example subset class counts: {'Astrocytes': 20000, 'B cells': 20000, 'Breast cancer cells': 20000, 'Cardiac muscle cells': 11608, 'Colon cancer cells': 20000}

Saved 20% subset to /hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/combined_meta_full_downsampled_no_bonecells_subset20pct.csv


## Coarse label, 20% subset of the downsampled dataset

In [3]:
file_path = "/hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/combined_meta_full_downsampled_no_bonecells_subset20pct.csv"
label_col = "cell_type_coarse"

counts = Counter()
total = 0

# Stream the CSV in chunks
for chunk in pd.read_csv(file_path, usecols=[label_col], chunksize=100_000):
    vals = chunk[label_col]
    counts.update(vals)
    total += len(vals)

# Print results
print("Counts and distribution:")
for label, count in counts.most_common():
    pct = count / total * 100
    print(f"{label}: {count:,} ({pct:.2f}%)")


Counts and distribution:
Cancer cells: 100,137 (23.89%)
Immune cells: 80,000 (19.09%)
Stromal cells: 53,394 (12.74%)
Glial cells: 49,447 (11.80%)
Muscle cells: 36,156 (8.63%)
Endothelial cells: 20,000 (4.77%)
Epithelial cells: 20,000 (4.77%)
Erythrocytes: 20,000 (4.77%)
Neurons: 20,000 (4.77%)
Stem and progenitor cells: 20,000 (4.77%)


## Fine label 20% subset of the downsampled dataset

In [4]:
file_path = "/hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/combined_meta_full_downsampled_no_bonecells_subset20pct.csv"
label_col = "cell_type"

counts = Counter()
total = 0

# Stream the CSV in chunks
for chunk in pd.read_csv(file_path, usecols=[label_col], chunksize=100_000):
    vals = chunk[label_col]
    counts.update(vals)
    total += len(vals)

# Print results
print("Counts and distribution:")
for label, count in counts.most_common():
    pct = count / total * 100
    print(f"{label}: {count:,} ({pct:.2f}%)")

Counts and distribution:
Astrocytes: 20,000 (4.77%)
B cells: 20,000 (4.77%)
Breast cancer cells: 20,000 (4.77%)
Colon cancer cells: 20,000 (4.77%)
Endothelial cells: 20,000 (4.77%)
Epithelial cells: 20,000 (4.77%)
Erythrocytes: 20,000 (4.77%)
Fibroblasts: 20,000 (4.77%)
Myeloid cells: 20,000 (4.77%)
NK cells: 20,000 (4.77%)
Neurons: 20,000 (4.77%)
Oligodendrocytes: 20,000 (4.77%)
Ovary cancer cells: 20,000 (4.77%)
Smooth muscle cells: 20,000 (4.77%)
Stem and progenitor cells: 20,000 (4.77%)
Stromal cells: 20,000 (4.77%)
T cells: 20,000 (4.77%)
Skin cancer cells: 14,605 (3.48%)
Lung cancer cells: 13,327 (3.18%)
Cardiac muscle cells: 11,608 (2.77%)
Pancreas cancer cells: 10,121 (2.41%)
Microglia: 9,447 (2.25%)
Pericytes: 9,368 (2.24%)
Skeletal muscle cells: 4,548 (1.09%)
Mesangial cells: 4,026 (0.96%)
Liver cancer cells: 2,084 (0.50%)


In [9]:
import pandas as pd
import json

subset_csv = "/hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/combined_meta_full_downsampled_no_bonecells_subset20pct.csv"
out_json   = "/hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/class_to_idx_coarse.json"
label_col  = "cell_type_coarse"

# Load subset
df = pd.read_csv(subset_csv)

# Get sorted unique classes
classes = sorted(df[label_col].unique())

# Build mapping {class_name: id}
class_to_idx = {cls: idx for idx, cls in enumerate(classes)}

# Save to JSON
with open(out_json, "w") as f:
    json.dump(class_to_idx, f, indent=2)

print("Saved mapping to", out_json)
print(class_to_idx)


Saved mapping to /hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/class_to_idx_coarse.json
{'Cancer cells': 0, 'Endothelial cells': 1, 'Epithelial cells': 2, 'Erythrocytes': 3, 'Glial cells': 4, 'Immune cells': 5, 'Muscle cells': 6, 'Neurons': 7, 'Stem and progenitor cells': 8, 'Stromal cells': 9}


## Train Test split of the 20% subset.

In [5]:
import os
import hashlib
import pandas as pd

in_csv   = "/hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/combined_meta_full_downsampled_no_bonecells_subset20pct.csv"  
train_csv = "/hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/20pct_train.csv"
val_csv   = "/hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/20pct_val.csv"
test_csv  = "/hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/20pct_test.csv"

label_col = "cell_type"
# Columns to build a stable, unique key per row. Adjust if needed.
key_cols = ["slide_id", "nucleus_id", "cell_id"]
seed = "23"  # string is fine

chunksize = 200_000
first = True

# Make sure parents exist
os.makedirs(os.path.dirname(train_csv), exist_ok=True)
os.makedirs(os.path.dirname(val_csv), exist_ok=True)
os.makedirs(os.path.dirname(test_csv), exist_ok=True)

# We only need the columns we will write
# If you want to keep all columns, set usecols=None
usecols = None  # keep all columns

def row_key(row):
    parts = []
    for c in key_cols:
        parts.append(str(row[c]) if c in row and pd.notna(row[c]) else "")
    return "|".join(parts)

def assign_bucket(row):
    # Deterministic hash in [0,1)
    k = row_key(row) + "|" + seed + "|" + str(row[label_col])
    h = hashlib.md5(k.encode("utf-8")).hexdigest()
    r = int(h, 16) / float(2**128)
    # 0.0 to 0.7999 -> train, 0.8 to 0.8999 -> val, else test
    if r < 0.8:
        return "train"
    elif r < 0.9:
        return "val"
    else:
        return "test"

# Counters
n_train = n_val = n_test = 0

for chunk in pd.read_csv(in_csv, chunksize=chunksize, usecols=usecols):
    # Safety: if any key col missing in the file, create empty string column
    for c in key_cols:
        if c not in chunk.columns:
            chunk[c] = ""

    # Compute split per row
    split = chunk.apply(assign_bucket, axis=1)

    # Write each split
    tmask = split == "train"
    vmask = split == "val"
    smask = split == "test"

    if tmask.any():
        chunk.loc[tmask].to_csv(train_csv, mode="w" if first else "a", index=False, header=first)
        n_train += int(tmask.sum())
    if vmask.any():
        chunk.loc[vmask].to_csv(val_csv, mode="w" if first else "a", index=False, header=first)
        n_val += int(vmask.sum())
    if smask.any():
        chunk.loc[smask].to_csv(test_csv, mode="w" if first else "a", index=False, header=first)
        n_test += int(smask.sum())

    first = False  # headers written after first batch

print(f"Train: {n_train:,} rows -> {train_csv}")
print(f"Val:   {n_val:,} rows -> {val_csv}")
print(f"Test:  {n_test:,} rows -> {test_csv}")



Train: 329,259 rows -> /hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/20pct_train.csv
Val:   53,375 rows -> /hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/20pct_val.csv
Test:  36,500 rows -> /hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/20pct_test.csv


In [6]:
file_path = "/hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/20pct_train.csv"
label_col = "cell_type"

counts = Counter()
total = 0

# Stream the CSV in chunks
for chunk in pd.read_csv(file_path, usecols=[label_col], chunksize=100_000):
    vals = chunk[label_col]
    counts.update(vals)
    total += len(vals)

# Print results
print("Counts and distribution:")
for label, count in counts.most_common():
    pct = count / total * 100
    print(f"{label}: {count:,} ({pct:.2f}%)")

Counts and distribution:
Colon cancer cells: 20,000 (6.07%)
Ovary cancer cells: 20,000 (6.07%)
Smooth muscle cells: 19,102 (5.80%)
Stromal cells: 19,030 (5.78%)
Breast cancer cells: 17,502 (5.32%)
Fibroblasts: 17,305 (5.26%)
Neurons: 16,973 (5.15%)
Endothelial cells: 16,281 (4.94%)
B cells: 15,859 (4.82%)
Myeloid cells: 15,808 (4.80%)
NK cells: 15,264 (4.64%)
Oligodendrocytes: 15,056 (4.57%)
Skin cancer cells: 14,605 (4.44%)
T cells: 14,239 (4.32%)
Epithelial cells: 14,230 (4.32%)
Erythrocytes: 13,725 (4.17%)
Astrocytes: 12,667 (3.85%)
Stem and progenitor cells: 11,778 (3.58%)
Pancreas cancer cells: 10,121 (3.07%)
Lung cancer cells: 8,943 (2.72%)
Microglia: 6,574 (2.00%)
Pericytes: 6,279 (1.91%)
Mesangial cells: 4,026 (1.22%)
Skeletal muscle cells: 3,182 (0.97%)
Cardiac muscle cells: 710 (0.22%)


In [7]:
file_path = "/hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/20pct_train.csv"
label_col = "cell_type_coarse"

counts = Counter()
total = 0

# Stream the CSV in chunks
for chunk in pd.read_csv(file_path, usecols=[label_col], chunksize=100_000):
    vals = chunk[label_col]
    counts.update(vals)
    total += len(vals)

# Print results
print("Counts and distribution:")
for label, count in counts.most_common():
    pct = count / total * 100
    print(f"{label}: {count:,} ({pct:.2f}%)")

Counts and distribution:
Cancer cells: 91,171 (27.69%)
Immune cells: 61,170 (18.58%)
Stromal cells: 46,640 (14.17%)
Glial cells: 34,297 (10.42%)
Muscle cells: 22,994 (6.98%)
Neurons: 16,973 (5.15%)
Endothelial cells: 16,281 (4.94%)
Epithelial cells: 14,230 (4.32%)
Erythrocytes: 13,725 (4.17%)
Stem and progenitor cells: 11,778 (3.58%)


## Original dataset with background

In [2]:
file_path = "/hpc/group/jilab/boxuan/data/combined_full/combined_meta_cell_type.csv"
label_col = "cell_type"

counts = Counter()
total = 0

# Stream the CSV in chunks
for chunk in pd.read_csv(file_path, usecols=[label_col], chunksize=100_000):
    vals = chunk[label_col]
    counts.update(vals)
    total += len(vals)

# Print results
print("Counts and distribution:")
for label, count in counts.most_common():
    pct = count / total * 100
    print(f"{label}: {count:,} ({pct:.2f}%)")

Counts and distribution:
Epithelial cells: 959,755 (12.61%)
Myeloid cells: 956,707 (12.57%)
Stem and progenitor cells: 827,997 (10.88%)
T cells: 807,476 (10.61%)
B cells: 699,126 (9.19%)
Endothelial cells: 615,478 (8.09%)
Fibroblasts: 376,857 (4.95%)
Smooth muscle cells: 309,461 (4.07%)
Neurons: 281,591 (3.70%)
Stromal cells: 247,968 (3.26%)
Erythrocytes: 201,938 (2.65%)
Cardiac muscle cells: 192,246 (2.53%)
Ovary cancer cells: 190,324 (2.50%)
Colon cancer cells: 176,040 (2.31%)
Oligodendrocytes: 95,514 (1.26%)
NK cells: 92,899 (1.22%)
Skin cancer cells: 73,025 (0.96%)
Mesangial cells: 68,076 (0.89%)
Osteoblasts: 64,389 (0.85%)
Lung cancer cells: 60,874 (0.80%)
Astrocytes: 56,141 (0.74%)
Pericytes: 54,472 (0.72%)
Pancreas cancer cells: 50,607 (0.66%)
Chondrocytes: 31,329 (0.41%)
Skeletal muscle cells: 30,656 (0.40%)
Schwann cells: 24,718 (0.32%)
Microglia: 21,921 (0.29%)
Liver cancer cells: 10,418 (0.14%)
Osteoclasts: 9,048 (0.12%)
Osteocytes: 8,236 (0.11%)
Bone marrow cancer cells: 5,

## Downsampling to 20:1 ratio.

In [1]:
import os
import csv
import random
import shutil
from pathlib import Path
from collections import Counter, defaultdict

import pandas as pd

# Inputs
in_csv = "/hpc/group/jilab/boxuan/data/combined_full/combined_meta_cell_type.csv"
out_csv = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled.csv"
tmp_dir = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/tmp_celltype_shards"
label_col = "cell_type"
keep_cols = [
    "slide_id", "cell_type_coarse", "cell_type",
    "raw_img_path", "mask_target_img_path", "mask_context_img_path",
    "nucleus_id", "cell_id",
]
chunksize = 200_000
cap = 100_000
min_keep = 5_000
seed = 23
random.seed(seed)

Path(tmp_dir).mkdir(parents=True, exist_ok=True)

# Pass 1. Count classes
counts = Counter()
for chunk in pd.read_csv(in_csv, usecols=[label_col], chunksize=chunksize):
    counts.update(chunk[label_col])

# Decide policy
policy = {}
for cls, n in counts.items():
    if n < min_keep:
        policy[cls] = ("drop", 0)
    elif n <= cap:
        policy[cls] = ("keep_all", n)
    else:
        policy[cls] = ("cap", cap)

num_drop = sum(1 for v in policy.values() if v[0] == "drop")
num_keep_all = sum(1 for v in policy.values() if v[0] == "keep_all")
num_cap = sum(1 for v in policy.values() if v[0] == "cap")
print(f"Class policy summary: drop={num_drop}, keep_all={num_keep_all}, cap={num_cap}")

# Prepare final file with header
with open(out_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(keep_cols)

# Helper to make safe shard file names
def shard_path_for_class(cls_name: str) -> Path:
    safe = "".join(ch if ch.isalnum() or ch in "._-" else "_" for ch in str(cls_name))
    return Path(tmp_dir) / f"class_{safe}.csv"

# Pass 2. Stream and write
# For keep_all classes, write directly to final
# For cap classes, write into class shard files
shard_writers = {}
shard_files = {}

def get_shard_writer(cls):
    if cls in shard_writers:
        return shard_writers[cls]
    p = shard_path_for_class(cls)
    f = open(p, "w", newline="")
    w = csv.writer(f)
    w.writerow(keep_cols)
    shard_files[cls] = f
    shard_writers[cls] = w
    return w

usecols = list(set(keep_cols + [label_col]))
total_rows = 0
kept_direct = 0
sharded = 0
dropped = 0

with open(out_csv, "a", newline="") as f_out:
    out_writer = csv.writer(f_out)
    for chunk in pd.read_csv(in_csv, usecols=usecols, chunksize=chunksize):
        total_rows += len(chunk)
        # restrict to the columns we want to write
        chunk = chunk[keep_cols]
        for row in chunk.itertuples(index=False, name=None):
            # row order matches keep_cols
            cls = row[keep_cols.index(label_col)]
            act, k = policy.get(cls, ("drop", 0))

            if act == "drop":
                dropped += 1
                continue
            if act == "keep_all":
                out_writer.writerow(row)
                kept_direct += 1
            else:
                # cap policy
                w = get_shard_writer(cls)
                w.writerow(row)
                sharded += 1

# Close all shard files
for f in shard_files.values():
    f.close()

print(f"Pass 2 done. Total rows seen {total_rows:,}. Wrote keep_all {kept_direct:,}. Sharded {sharded:,}. Dropped {dropped:,}.")

# Pass 3. For each capped class shard, do reservoir sample of size cap, append to final
def reservoir_sample_from_csv(csv_path: Path, k: int, header=True):
    """Return a list of up to k rows from csv_path using Algorithm R."""
    sample = []
    with open(csv_path, "r", newline="") as f:
        reader = csv.reader(f)
        if header:
            next(reader, None)
        seen = 0
        for row in reader:
            seen += 1
            if len(sample) < k:
                sample.append(row)
            else:
                j = random.randint(1, seen)
                if j <= k:
                    sample[j - 1] = row
    return sample

added_from_shards = 0
with open(out_csv, "a", newline="") as f_out:
    out_writer = csv.writer(f_out)
    for cls, (act, k) in policy.items():
        if act != "cap":
            continue
        shard_path = shard_path_for_class(cls)
        if not shard_path.exists():
            # No rows ended up in this shard. Skip.
            continue
        # For safety, compute target k as min(cap, actual rows)
        # This also protects against off by ones
        # Count rows quickly
        with open(shard_path, "r") as f:
            rows_in_file = sum(1 for _ in f) - 1  # minus header
        target_k = min(k, rows_in_file)
        if target_k <= 0:
            continue
        sample_rows = reservoir_sample_from_csv(shard_path, target_k, header=True)
        out_writer.writerows(sample_rows)
        added_from_shards += len(sample_rows)
        print(f"Appended {len(sample_rows):,} rows for class {cls}")

print(f"Done. Final file: {out_csv}")
print(f"Rows appended from shards {added_from_shards:,}. Now you can remove temp dir if you want: {tmp_dir}")

# Optional cleanup
# shutil.rmtree(tmp_dir)


Class policy summary: drop=4, keep_all=17, cap=14
Pass 2 done. Total rows seen 7,610,170. Wrote keep_all 758,085. Sharded 6,842,964. Dropped 9,121.
Appended 100,000 rows for class Myeloid cells
Appended 100,000 rows for class Smooth muscle cells
Appended 100,000 rows for class Fibroblasts
Appended 100,000 rows for class Epithelial cells
Appended 100,000 rows for class Endothelial cells
Appended 100,000 rows for class T cells
Appended 100,000 rows for class B cells
Appended 100,000 rows for class Cardiac muscle cells
Appended 100,000 rows for class Stromal cells
Appended 100,000 rows for class Erythrocytes
Appended 100,000 rows for class Neurons
Appended 100,000 rows for class Ovary cancer cells
Appended 100,000 rows for class Stem and progenitor cells
Appended 100,000 rows for class Colon cancer cells
Done. Final file: /hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled.csv
Rows appended from shards 1,400,000. Now you can remove temp dir if you want: /hpc/gr

In [2]:
file_path = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled.csv"
label_col = "cell_type"

counts = Counter()
total = 0

# Stream the CSV in chunks
for chunk in pd.read_csv(file_path, usecols=[label_col], chunksize=100_000):
    vals = chunk[label_col]
    counts.update(vals)
    total += len(vals)

# Print results
print("Counts and distribution:")
for label, count in counts.most_common():
    pct = count / total * 100
    print(f"{label}: {count:,} ({pct:.2f}%)")

Counts and distribution:
Myeloid cells: 100,000 (4.63%)
Smooth muscle cells: 100,000 (4.63%)
Fibroblasts: 100,000 (4.63%)
Epithelial cells: 100,000 (4.63%)
Endothelial cells: 100,000 (4.63%)
T cells: 100,000 (4.63%)
B cells: 100,000 (4.63%)
Cardiac muscle cells: 100,000 (4.63%)
Stromal cells: 100,000 (4.63%)
Erythrocytes: 100,000 (4.63%)
Neurons: 100,000 (4.63%)
Ovary cancer cells: 100,000 (4.63%)
Stem and progenitor cells: 100,000 (4.63%)
Colon cancer cells: 100,000 (4.63%)
Oligodendrocytes: 95,514 (4.43%)
NK cells: 92,899 (4.30%)
Skin cancer cells: 73,025 (3.38%)
Mesangial cells: 68,076 (3.15%)
Osteoblasts: 64,389 (2.98%)
Lung cancer cells: 60,874 (2.82%)
Astrocytes: 56,141 (2.60%)
Pericytes: 54,472 (2.52%)
Pancreas cancer cells: 50,607 (2.34%)
Chondrocytes: 31,329 (1.45%)
Skeletal muscle cells: 30,656 (1.42%)
Schwann cells: 24,718 (1.15%)
Microglia: 21,921 (1.02%)
Liver cancer cells: 10,418 (0.48%)
Osteoclasts: 9,048 (0.42%)
Osteocytes: 8,236 (0.38%)
Bone marrow cancer cells: 5,762 

## Remove Bone cells

In [16]:
import pandas as pd

in_csv = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled.csv"
out_csv = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled_no_bonecells.csv"

label_col = "cell_type"
# list of classes to drop
drop_classes = [
    "Bone marrow cancer cells",
    "Osteocytes",
    "Osteoblasts",
    "Osteoclasts",
    "Chondrocytes"
    # add more labels here...
]

# stream in chunks and filter
chunksize = 200_000
first = True
kept = 0
dropped = 0

for chunk in pd.read_csv(in_csv, chunksize=chunksize):
    mask = ~chunk[label_col].isin(drop_classes)
    kept += mask.sum()
    dropped += (~mask).sum()
    chunk = chunk[mask]
    chunk.to_csv(out_csv, mode="w" if first else "a", index=False, header=first)
    first = False

print(f"Done. Wrote {out_csv}")
print(f"Kept rows: {kept:,}")
print(f"Dropped rows: {dropped:,}")


Done. Wrote /hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled_no_bonecells.csv
Kept rows: 2,039,321
Dropped rows: 118,764


In [17]:
file_path = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled_no_bonecells.csv"
label_col = "cell_type"

counts = Counter()
total = 0

# Stream the CSV in chunks
for chunk in pd.read_csv(file_path, usecols=[label_col], chunksize=100_000):
    vals = chunk[label_col]
    counts.update(vals)
    total += len(vals)

# Print results
print("Counts and distribution:")
for label, count in counts.most_common():
    pct = count / total * 100
    print(f"{label}: {count:,} ({pct:.2f}%)")

Counts and distribution:
Myeloid cells: 100,000 (4.90%)
Smooth muscle cells: 100,000 (4.90%)
Fibroblasts: 100,000 (4.90%)
Epithelial cells: 100,000 (4.90%)
Endothelial cells: 100,000 (4.90%)
T cells: 100,000 (4.90%)
B cells: 100,000 (4.90%)
Cardiac muscle cells: 100,000 (4.90%)
Stromal cells: 100,000 (4.90%)
Erythrocytes: 100,000 (4.90%)
Neurons: 100,000 (4.90%)
Ovary cancer cells: 100,000 (4.90%)
Stem and progenitor cells: 100,000 (4.90%)
Colon cancer cells: 100,000 (4.90%)
Oligodendrocytes: 95,514 (4.68%)
NK cells: 92,899 (4.56%)
Skin cancer cells: 73,025 (3.58%)
Mesangial cells: 68,076 (3.34%)
Lung cancer cells: 60,874 (2.99%)
Astrocytes: 56,141 (2.75%)
Pericytes: 54,472 (2.67%)
Pancreas cancer cells: 50,607 (2.48%)
Skeletal muscle cells: 30,656 (1.50%)
Schwann cells: 24,718 (1.21%)
Microglia: 21,921 (1.07%)
Liver cancer cells: 10,418 (0.51%)


## Include COARSE LABEL

In [10]:
import os
import json
import pandas as pd
from collections import Counter

# --- paths ---
in_csv  = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled_no_bonecells.csv"
out_csv = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled_no_bonecells_withCoarse.csv"
mapping_json = "/hpc/group/jilab/rz179/cellpt/combined/fine_to_coarse_map.json"   # <- update to your json path

# --- load mapping ---
with open(mapping_json, "r") as f:
    fine_to_coarse = json.load(f)

cell_col = "cell_type"
coarse_col = "cell_type_coarse"
chunksize = 200_000   # adjust to your memory

# --- prepare output ---
if os.path.exists(out_csv):
    os.remove(out_csv)

unmapped = Counter()
row_count = 0
updated_count = 0

for i, chunk in enumerate(pd.read_csv(in_csv, chunksize=chunksize)):
    # map fine -> coarse
    mapped = chunk[cell_col].map(fine_to_coarse)

    # track unmapped
    missing_mask = mapped.isna()
    if missing_mask.any():
        for lbl, cnt in chunk.loc[missing_mask, cell_col].value_counts().items():
            unmapped[lbl] += int(cnt)

    # overwrite or create coarse column
    chunk[coarse_col] = mapped
    updated_count += int(chunk[coarse_col].notna().sum())
    row_count += len(chunk)

    # write out
    mode = "w" if i == 0 else "a"
    header = (i == 0)
    chunk.to_csv(out_csv, index=False, mode=mode, header=header)

    print(f"Wrote chunk {i}, rows {len(chunk)}, total {row_count}")

print("===================================================")
print(f"Total rows processed: {row_count}")
print(f"Rows mapped successfully: {updated_count}")
if unmapped:
    print("Unmapped labels found:")
    for k, v in unmapped.most_common():
        print(f"  {k}: {v}")
    print("Add them to your JSON if needed and rerun.")



Wrote chunk 0, rows 200000, total 200000
Wrote chunk 1, rows 200000, total 400000
Wrote chunk 2, rows 200000, total 600000
Wrote chunk 3, rows 200000, total 800000
Wrote chunk 4, rows 200000, total 1000000
Wrote chunk 5, rows 200000, total 1200000
Wrote chunk 6, rows 200000, total 1400000
Wrote chunk 7, rows 200000, total 1600000
Wrote chunk 8, rows 200000, total 1800000
Wrote chunk 9, rows 200000, total 2000000
Wrote chunk 10, rows 39321, total 2039321
Total rows processed: 2039321
Rows mapped successfully: 2039321


In [11]:
file_path = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled_no_bonecells_withCoarse.csv"
label_col = "cell_type_coarse"

counts = Counter()
total = 0

# Stream the CSV in chunks
for chunk in pd.read_csv(file_path, usecols=[label_col], chunksize=100_000):
    vals = chunk[label_col]
    counts.update(vals)
    total += len(vals)

# Print results
print("Counts and distribution:")
for label, count in counts.most_common():
    pct = count / total * 100
    print(f"{label}: {count:,} ({pct:.2f}%)")

Counts and distribution:
Cancer cells: 394,924 (19.37%)
Immune cells: 392,899 (19.27%)
Stromal cells: 322,548 (15.82%)
Muscle cells: 230,656 (11.31%)
Glial cells: 198,294 (9.72%)
Epithelial cells: 100,000 (4.90%)
Endothelial cells: 100,000 (4.90%)
Erythrocytes: 100,000 (4.90%)
Neurons: 100,000 (4.90%)
Stem and progenitor cells: 100,000 (4.90%)


## Train Test Split based on fine cell_type

In [14]:
import csv, math, random
from collections import Counter, defaultdict

in_csv   = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled_no_bonecells_withCoarse.csv"
out_csv  = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled_no_bonecells_withCoarse_20pct.csv"
label_col = "cell_type"
subset_frac = 0.20
seed = 23

random.seed(seed)

def count_classes(path, label_col):
    counts = Counter()
    with open(path, "r", newline="") as f:
        reader = csv.reader(f)
        header = next(reader)
        y_idx = header.index(label_col)
        for row in reader:
            counts[row[y_idx]] += 1
    return counts, header, y_idx

def compute_targets(counts, frac):
    float_counts = {c: counts[c] * frac for c in counts}
    floor_counts = {c: int(math.floor(v)) for c, v in float_counts.items()}
    remainder = int(round(sum(float_counts.values()))) - sum(floor_counts.values())
    # distribute remainder based on largest fractional part
    frac_parts = sorted(
        ((c, float_counts[c] - floor_counts[c]) for c in counts),
        key=lambda x: x[1],
        reverse=True,
    )
    targets = floor_counts.copy()
    for i in range(remainder):
        targets[frac_parts[i][0]] += 1
    return targets

def stream_subset(in_csv, out_csv, header, y_idx, targets):
    remaining_in_class = defaultdict(int)
    with open(in_csv, "r", newline="") as f:
        reader = csv.reader(f)
        _ = next(reader)
        for row in reader:
            remaining_in_class[row[y_idx]] += 1

    rem_targets = targets.copy()

    with open(in_csv, "r", newline="") as f_in, open(out_csv, "w", newline="") as f_out:
        reader = csv.reader(f_in)
        writer = csv.writer(f_out)
        writer.writerow(header)
        _ = next(reader)
        for row in reader:
            c = row[y_idx]
            if rem_targets.get(c, 0) <= 0:
                remaining_in_class[c] -= 1
                continue
            prob = rem_targets[c] / remaining_in_class[c]
            if random.random() <= prob:
                writer.writerow(row)
                rem_targets[c] -= 1
            remaining_in_class[c] -= 1

def summarize(path, label_col):
    counts = Counter()
    total = 0
    with open(path, "r", newline="") as f:
        reader = csv.reader(f)
        header = next(reader)
        y_idx = header.index(label_col)
        for row in reader:
            counts[row[y_idx]] += 1
            total += 1
    dist = {k: v / total for k, v in counts.items()} if total > 0 else {}
    return total, counts, dist

# Run
counts, header, y_idx = count_classes(in_csv, label_col)
targets = compute_targets(counts, subset_frac)
stream_subset(in_csv, out_csv, header, y_idx, targets)
total, counts_sub, dist_sub = summarize(out_csv, label_col)

print("Original total:", sum(counts.values()))
print("Subset total:", total)
print("Example original class counts:", dict(list(counts.items())[:5]))
print("Example subset class counts:", dict(list(counts_sub.items())[:5]))
print(f"\nSaved 20% subset to {out_csv}")


Original total: 2039321
Subset total: 407864
Example original class counts: {'Pericytes': 54472, 'NK cells': 92899, 'Lung cancer cells': 60874, 'Skeletal muscle cells': 30656, 'Schwann cells': 24718}
Example subset class counts: {'Pericytes': 10894, 'NK cells': 18580, 'Lung cancer cells': 12175, 'Skeletal muscle cells': 6131, 'Schwann cells': 4944}

Saved 20% subset to /hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled_no_bonecells_withCoarse_20pct.csv


In [15]:
import os
import hashlib
import pandas as pd

in_csv   = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled_no_bonecells_withCoarse_20pct.csv"   #our base dataset. 20:1
train_csv = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/20pct_train.csv"
val_csv   = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/20pct_val.csv"
test_csv  = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/20pct_test.csv"

label_col = "cell_type"
# Columns to build a stable, unique key per row. Adjust if needed.
key_cols = ["slide_id", "nucleus_id", "cell_id"]
seed = "23"  # string is fine

chunksize = 200_000
first = True

# Make sure parents exist
os.makedirs(os.path.dirname(train_csv), exist_ok=True)
os.makedirs(os.path.dirname(val_csv), exist_ok=True)
os.makedirs(os.path.dirname(test_csv), exist_ok=True)

# We only need the columns we will write
# If you want to keep all columns, set usecols=None
usecols = None  # keep all columns

def row_key(row):
    parts = []
    for c in key_cols:
        parts.append(str(row[c]) if c in row and pd.notna(row[c]) else "")
    return "|".join(parts)

def assign_bucket(row):
    # Deterministic hash in [0,1)
    k = row_key(row) + "|" + seed + "|" + str(row[label_col])
    h = hashlib.md5(k.encode("utf-8")).hexdigest()
    r = int(h, 16) / float(2**128)
    # 0.0 to 0.7999 -> train, 0.8 to 0.8999 -> val, else test
    if r < 0.8:
        return "train"
    elif r < 0.9:
        return "val"
    else:
        return "test"

# Counters
n_train = n_val = n_test = 0

for chunk in pd.read_csv(in_csv, chunksize=chunksize, usecols=usecols):
    # Safety: if any key col missing in the file, create empty string column
    for c in key_cols:
        if c not in chunk.columns:
            chunk[c] = ""

    # Compute split per row
    split = chunk.apply(assign_bucket, axis=1)

    # Write each split
    tmask = split == "train"
    vmask = split == "val"
    smask = split == "test"

    if tmask.any():
        chunk.loc[tmask].to_csv(train_csv, mode="w" if first else "a", index=False, header=first)
        n_train += int(tmask.sum())
    if vmask.any():
        chunk.loc[vmask].to_csv(val_csv, mode="w" if first else "a", index=False, header=first)
        n_val += int(vmask.sum())
    if smask.any():
        chunk.loc[smask].to_csv(test_csv, mode="w" if first else "a", index=False, header=first)
        n_test += int(smask.sum())

    first = False  # headers written after first batch

print(f"Train: {n_train:,} rows -> {train_csv}")
print(f"Val:   {n_val:,} rows -> {val_csv}")
print(f"Test:  {n_test:,} rows -> {test_csv}")



Train: 325,854 rows -> /hpc/group/jilab/rz179/cellpt/combined/withBackground/20pct_train.csv
Val:   40,838 rows -> /hpc/group/jilab/rz179/cellpt/combined/withBackground/20pct_val.csv
Test:  41,172 rows -> /hpc/group/jilab/rz179/cellpt/combined/withBackground/20pct_test.csv


In [16]:
file_path = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/20pct_train.csv"
label_col = "cell_type"

counts = Counter()
total = 0

# Stream the CSV in chunks
for chunk in pd.read_csv(file_path, usecols=[label_col], chunksize=100_000):
    vals = chunk[label_col]
    counts.update(vals)
    total += len(vals)

# Print results
print("Counts and distribution:")
for label, count in counts.most_common():
    pct = count / total * 100
    print(f"{label}: {count:,} ({pct:.2f}%)")

Counts and distribution:
Ovary cancer cells: 16,036 (4.92%)
Neurons: 16,019 (4.92%)
Fibroblasts: 16,017 (4.92%)
Myeloid cells: 16,011 (4.91%)
Endothelial cells: 16,008 (4.91%)
Cardiac muscle cells: 16,003 (4.91%)
B cells: 15,999 (4.91%)
Smooth muscle cells: 15,963 (4.90%)
Epithelial cells: 15,941 (4.89%)
Stem and progenitor cells: 15,934 (4.89%)
T cells: 15,933 (4.89%)
Colon cancer cells: 15,930 (4.89%)
Erythrocytes: 15,925 (4.89%)
Stromal cells: 15,898 (4.88%)
Oligodendrocytes: 15,286 (4.69%)
NK cells: 14,890 (4.57%)
Skin cancer cells: 11,750 (3.61%)
Mesangial cells: 10,851 (3.33%)
Lung cancer cells: 9,638 (2.96%)
Astrocytes: 8,946 (2.75%)
Pericytes: 8,688 (2.67%)
Pancreas cancer cells: 8,119 (2.49%)
Skeletal muscle cells: 4,897 (1.50%)
Schwann cells: 4,000 (1.23%)
Microglia: 3,494 (1.07%)
Liver cancer cells: 1,678 (0.51%)


In [19]:
import pandas as pd
import json

subset_csv = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled_no_bonecells_withCoarse_20pct.csv"
out_json   = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/class_to_idx_coarse.json"
label_col  = "cell_type_coarse"

# Load subset
df = pd.read_csv(subset_csv)

# Get sorted unique classes
classes = sorted(df[label_col].unique())

# Build mapping {class_name: id}
class_to_idx = {cls: idx for idx, cls in enumerate(classes)}

# Save to JSON
with open(out_json, "w") as f:
    json.dump(class_to_idx, f, indent=2)

print("Saved mapping to", out_json)
print(class_to_idx)


Saved mapping to /hpc/group/jilab/rz179/cellpt/combined/withBackground/class_to_idx_coarse.json
{'Cancer cells': 0, 'Endothelial cells': 1, 'Epithelial cells': 2, 'Erythrocytes': 3, 'Glial cells': 4, 'Immune cells': 5, 'Muscle cells': 6, 'Neurons': 7, 'Stem and progenitor cells': 8, 'Stromal cells': 9}


## Remove Mouse

In [1]:
import pandas as pd

def is_human_tissue(slide_id):
    """
    Determine if a slide_id represents human tissue.
    Returns True for human, False for mouse.
    """
    slide_id_lower = slide_id.lower()
    
    # Mouse identifiers to exclude
    mouse_patterns = [
        'mouse',           # Contains "mouse"
        '_m',             # Starts with m after underscore (like Xenium_V1_mFemur)
        'xenium_v1_m',    # Specific pattern for mouse samples
        'pup',            # Mouse pup samples
        'tgcrnd8',        # Transgenic mouse model
        'wildtype'        # Wildtype mouse samples
    ]
    
    # Check if any mouse pattern is found
    for pattern in mouse_patterns:
        if pattern in slide_id_lower:
            return False
    
    # Additional specific checks
    if slide_id_lower.startswith('xenium_prime_mouse'):
        return False
        
    # If none of the mouse patterns match, assume it's human
    return True

def filter_human_tissues(input_file, output_file, chunksize=5000):
    """
    Filter CSV to keep only human tissues, processing in chunks.
    """
    print(f"Filtering human tissues from: {input_file}")
    print(f"Output will be saved to: {output_file}")
    print(f"Processing in chunks of {chunksize} rows...")
    
    first_chunk = True
    total_rows_processed = 0
    human_rows_kept = 0
    mouse_rows_removed = 0
    
    try:
        for chunk_num, chunk in enumerate(pd.read_csv(input_file, chunksize=chunksize), 1):
            print(f"Processing chunk {chunk_num}...")
            
            # Filter for human tissues
            human_mask = chunk['slide_id'].apply(is_human_tissue)
            human_chunk = chunk[human_mask]
            
            # Update counters
            total_rows_processed += len(chunk)
            human_rows_kept += len(human_chunk)
            mouse_rows_removed += len(chunk) - len(human_chunk)
            
            # Write to output file
            if first_chunk:
                # Write header for first chunk
                human_chunk.to_csv(output_file, index=False, mode='w')
                first_chunk = False
                print(f"  Created output file with {len(human_chunk)} rows")
            else:
                # Append without header for subsequent chunks
                human_chunk.to_csv(output_file, index=False, mode='a', header=False)
                print(f"  Added {len(human_chunk)} rows to output")
            
            print(f"  Progress: {len(human_chunk)}/{len(chunk)} rows kept in this chunk")
            print(f"  Running totals: {human_rows_kept:,} kept, {mouse_rows_removed:,} removed")
            
        print(f"\n{'='*50}")
        print(f"FILTERING COMPLETE!")
        print(f"{'='*50}")
        print(f"Total rows processed: {total_rows_processed:,}")
        print(f"Human rows kept: {human_rows_kept:,}")
        print(f"Mouse rows removed: {mouse_rows_removed:,}")
        print(f"Percentage kept: {(human_rows_kept/total_rows_processed)*100:.1f}%")
        print(f"Output saved to: {output_file}")
        
        return True
        
    except Exception as e:
        print(f"Error during filtering: {e}")
        return False

# File paths
input_file = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled_no_bonecells_withCoarse.csv"
output_file = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_human_only.csv"  # Save to work directory

print("Starting the filtering process...")
print("This will:")
print("1. Read the original CSV in chunks")
print("2. Keep only rows where slide_id represents human tissues")
print("3. Remove all mouse tissue rows")
print("4. Save the filtered data to a new CSV")
print()

# Try with progressively smaller chunk sizes if needed
chunk_sizes = [5000, 1000, 100]

print("Attempting filtering with different chunk sizes...")

success = False
for chunk_size in chunk_sizes:
    print(f"\nTrying with chunk size: {chunk_size}")
    try:
        success = filter_human_tissues(input_file, output_file, chunksize=chunk_size)
        if success:
            break
    except Exception as e:
        print(f"Failed with chunk size {chunk_size}: {e}")
        continue

if success:
    print("\n✅ Filtering completed successfully!")
    print(f"Your human-only dataset is ready at: {output_file}")
else:
    print("\n❌ All Python methods failed.")
    print("Trying command-line approach instead...")
    
    # Command line fallback
    print("\nCOMMAND LINE FALLBACK:")
    print("Run these commands in your terminal:")
    print(f"cd /work/rz179")
    print(f"# First, find the slide_id column number:")
    print(f"head -1 '{input_file}' | tr ',' '\\n' | nl")
    print(f"# Then filter (replace X with slide_id column number):")
    print(f"awk -F',' 'NR==1 {{print}} NR>1 && $X !~ /mouse|Mouse|pup|TgCRND8|wildtype|mFemur/ {{print}}' '{input_file}' > combined_meta_human_only.csv")

Starting the filtering process...
This will:
1. Read the original CSV in chunks
2. Keep only rows where slide_id represents human tissues
3. Remove all mouse tissue rows
4. Save the filtered data to a new CSV

Attempting filtering with different chunk sizes...

Trying with chunk size: 5000
Filtering human tissues from: /hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled_no_bonecells_withCoarse.csv
Output will be saved to: /hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_human_only.csv
Processing in chunks of 5000 rows...
Processing chunk 1...
  Created output file with 5000 rows
  Progress: 5000/5000 rows kept in this chunk
  Running totals: 5,000 kept, 0 removed
Processing chunk 2...
  Added 5000 rows to output
  Progress: 5000/5000 rows kept in this chunk
  Running totals: 10,000 kept, 0 removed
Processing chunk 3...
  Added 5000 rows to output
  Progress: 5000/5000 rows kept in this chunk
  Running totals: 15,000 kept, 0 removed
Process

## Remove tail classes from HUMAN ONLY, that has less than 5000 samples

In [2]:
import os
import csv
import random
import shutil
from pathlib import Path
from collections import Counter, defaultdict

import pandas as pd

# Inputs
in_csv = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_human_only.csv"
out_csv = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_human_only_20to1.csv"
tmp_dir = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/tmp_celltype_shards"
label_col = "cell_type"
keep_cols = [
    "slide_id", "cell_type_coarse", "cell_type",
    "raw_img_path", "mask_target_img_path", "mask_context_img_path",
    "nucleus_id", "cell_id",
]
chunksize = 200_000
cap = 100_000
min_keep = 5_000
seed = 23
random.seed(seed)

Path(tmp_dir).mkdir(parents=True, exist_ok=True)

# Pass 1. Count classes
counts = Counter()
for chunk in pd.read_csv(in_csv, usecols=[label_col], chunksize=chunksize):
    counts.update(chunk[label_col])

# Decide policy
policy = {}
for cls, n in counts.items():
    if n < min_keep:
        policy[cls] = ("drop", 0)
    elif n <= cap:
        policy[cls] = ("keep_all", n)
    else:
        policy[cls] = ("cap", cap)

num_drop = sum(1 for v in policy.values() if v[0] == "drop")
num_keep_all = sum(1 for v in policy.values() if v[0] == "keep_all")
num_cap = sum(1 for v in policy.values() if v[0] == "cap")
print(f"Class policy summary: drop={num_drop}, keep_all={num_keep_all}, cap={num_cap}")

# Prepare final file with header
with open(out_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(keep_cols)

# Helper to make safe shard file names
def shard_path_for_class(cls_name: str) -> Path:
    safe = "".join(ch if ch.isalnum() or ch in "._-" else "_" for ch in str(cls_name))
    return Path(tmp_dir) / f"class_{safe}.csv"

# Pass 2. Stream and write
# For keep_all classes, write directly to final
# For cap classes, write into class shard files
shard_writers = {}
shard_files = {}

def get_shard_writer(cls):
    if cls in shard_writers:
        return shard_writers[cls]
    p = shard_path_for_class(cls)
    f = open(p, "w", newline="")
    w = csv.writer(f)
    w.writerow(keep_cols)
    shard_files[cls] = f
    shard_writers[cls] = w
    return w

usecols = list(set(keep_cols + [label_col]))
total_rows = 0
kept_direct = 0
sharded = 0
dropped = 0

with open(out_csv, "a", newline="") as f_out:
    out_writer = csv.writer(f_out)
    for chunk in pd.read_csv(in_csv, usecols=usecols, chunksize=chunksize):
        total_rows += len(chunk)
        # restrict to the columns we want to write
        chunk = chunk[keep_cols]
        for row in chunk.itertuples(index=False, name=None):
            # row order matches keep_cols
            cls = row[keep_cols.index(label_col)]
            act, k = policy.get(cls, ("drop", 0))

            if act == "drop":
                dropped += 1
                continue
            if act == "keep_all":
                out_writer.writerow(row)
                kept_direct += 1
            else:
                # cap policy
                w = get_shard_writer(cls)
                w.writerow(row)
                sharded += 1

# Close all shard files
for f in shard_files.values():
    f.close()

print(f"Pass 2 done. Total rows seen {total_rows:,}. Wrote keep_all {kept_direct:,}. Sharded {sharded:,}. Dropped {dropped:,}.")

# Pass 3. For each capped class shard, do reservoir sample of size cap, append to final
def reservoir_sample_from_csv(csv_path: Path, k: int, header=True):
    """Return a list of up to k rows from csv_path using Algorithm R."""
    sample = []
    with open(csv_path, "r", newline="") as f:
        reader = csv.reader(f)
        if header:
            next(reader, None)
        seen = 0
        for row in reader:
            seen += 1
            if len(sample) < k:
                sample.append(row)
            else:
                j = random.randint(1, seen)
                if j <= k:
                    sample[j - 1] = row
    return sample

added_from_shards = 0
with open(out_csv, "a", newline="") as f_out:
    out_writer = csv.writer(f_out)
    for cls, (act, k) in policy.items():
        if act != "cap":
            continue
        shard_path = shard_path_for_class(cls)
        if not shard_path.exists():
            # No rows ended up in this shard. Skip.
            continue
        # For safety, compute target k as min(cap, actual rows)
        # This also protects against off by ones
        # Count rows quickly
        with open(shard_path, "r") as f:
            rows_in_file = sum(1 for _ in f) - 1  # minus header
        target_k = min(k, rows_in_file)
        if target_k <= 0:
            continue
        sample_rows = reservoir_sample_from_csv(shard_path, target_k, header=True)
        out_writer.writerows(sample_rows)
        added_from_shards += len(sample_rows)
        print(f"Appended {len(sample_rows):,} rows for class {cls}")

print(f"Done. Final file: {out_csv}")
print(f"Rows appended from shards {added_from_shards:,}. Now you can remove temp dir if you want: {tmp_dir}")

# Optional cleanup
# shutil.rmtree(tmp_dir)

Class policy summary: drop=2, keep_all=20, cap=0
Pass 2 done. Total rows seen 1,100,356. Wrote keep_all 1,097,184. Sharded 0. Dropped 3,172.
Done. Final file: /hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_human_only_20to1.csv
Rows appended from shards 0. Now you can remove temp dir if you want: /hpc/group/jilab/rz179/cellpt/combined/withBackground/tmp_celltype_shards


# Train test split for full human only dataset. 

In [3]:
import os
import hashlib
import pandas as pd

in_csv   = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_human_only_20to1.csv"  
train_csv = "/hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/full_train.csv"
val_csv   = "/hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/full_val.csv"
test_csv  = "/hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/full_test.csv"

label_col = "cell_type"
# Columns to build a stable, unique key per row. Adjust if needed.
key_cols = ["slide_id", "nucleus_id", "cell_id"]
seed = "23"  # string is fine

chunksize = 200_000
first = True

# Make sure parents exist
os.makedirs(os.path.dirname(train_csv), exist_ok=True)
os.makedirs(os.path.dirname(val_csv), exist_ok=True)
os.makedirs(os.path.dirname(test_csv), exist_ok=True)

# We only need the columns we will write
# If you want to keep all columns, set usecols=None
usecols = None  # keep all columns

def row_key(row):
    parts = []
    for c in key_cols:
        parts.append(str(row[c]) if c in row and pd.notna(row[c]) else "")
    return "|".join(parts)

def assign_bucket(row):
    # Deterministic hash in [0,1)
    k = row_key(row) + "|" + seed + "|" + str(row[label_col])
    h = hashlib.md5(k.encode("utf-8")).hexdigest()
    r = int(h, 16) / float(2**128)
    # 0.0 to 0.7999 -> train, 0.8 to 0.8999 -> val, else test
    if r < 0.8:
        return "train"
    elif r < 0.9:
        return "val"
    else:
        return "test"

# Counters
n_train = n_val = n_test = 0

for chunk in pd.read_csv(in_csv, chunksize=chunksize, usecols=usecols):
    # Safety: if any key col missing in the file, create empty string column
    for c in key_cols:
        if c not in chunk.columns:
            chunk[c] = ""

    # Compute split per row
    split = chunk.apply(assign_bucket, axis=1)

    # Write each split
    tmask = split == "train"
    vmask = split == "val"
    smask = split == "test"

    if tmask.any():
        chunk.loc[tmask].to_csv(train_csv, mode="w" if first else "a", index=False, header=first)
        n_train += int(tmask.sum())
    if vmask.any():
        chunk.loc[vmask].to_csv(val_csv, mode="w" if first else "a", index=False, header=first)
        n_val += int(vmask.sum())
    if smask.any():
        chunk.loc[smask].to_csv(test_csv, mode="w" if first else "a", index=False, header=first)
        n_test += int(smask.sum())

    first = False  # headers written after first batch

print(f"Train: {n_train:,} rows -> {train_csv}")
print(f"Val:   {n_val:,} rows -> {val_csv}")
print(f"Test:  {n_test:,} rows -> {test_csv}")

Train: 877,217 rows -> /hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/full_train.csv
Val:   109,864 rows -> /hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/full_val.csv
Test:  110,103 rows -> /hpc/group/jilab/rz179/cellpt/combined/54s/semi_balanced/full_test.csv


In [4]:
file_path = "/hpc/group/jilab/rz179/cellpt/combined/semi_balanced/full_train.csv"
label_col = "cell_type"

counts = Counter()
total = 0

# Stream the CSV in chunks
for chunk in pd.read_csv(file_path, usecols=[label_col], chunksize=100_000):
    vals = chunk[label_col]
    counts.update(vals)
    total += len(vals)

# Print results
print("Counts and distribution:")
for label, count in counts.most_common():
    pct = count / total * 100
    print(f"{label}: {count:,} ({pct:.2f}%)")

Counts and distribution:
Ovary cancer cells: 80,115 (9.13%)
Colon cancer cells: 80,115 (9.13%)
T cells: 71,680 (8.17%)
NK cells: 71,673 (8.17%)
B cells: 66,106 (7.54%)
Stem and progenitor cells: 61,619 (7.02%)
Fibroblasts: 60,313 (6.88%)
Myeloid cells: 58,714 (6.69%)
Epithelial cells: 53,236 (6.07%)
Endothelial cells: 49,530 (5.65%)
Lung cancer cells: 48,547 (5.53%)
Smooth muscle cells: 45,034 (5.13%)
Pancreas cancer cells: 40,418 (4.61%)
Oligodendrocytes: 24,559 (2.80%)
Pericytes: 21,909 (2.50%)
Stromal cells: 19,243 (2.19%)
Liver cancer cells: 8,305 (0.95%)
Microglia: 6,074 (0.69%)
Astrocytes: 5,949 (0.68%)
Neurons: 4,078 (0.46%)


In [4]:
file_path = "/hpc/group/jilab/rz179/cellpt/combined/subset_per_slide_two_eval_splits/subset_all.csv"
label_col = "cell_type"

counts = Counter()
total = 0

# Stream the CSV in chunks
for chunk in pd.read_csv(file_path, usecols=[label_col], chunksize=100_000):
    vals = chunk[label_col]
    counts.update(vals)
    total += len(vals)

# Print results
print("Counts and distribution:")
for label, count in counts.most_common():
    pct = count / total * 100
    print(f"{label}: {count:,} ({pct:.2f}%)")

Counts and distribution:
Smooth muscle cells: 30,001 (8.57%)
Fibroblasts: 30,001 (8.57%)
Oligodendrocytes: 23,014 (6.58%)
Colon cancer cells: 22,334 (6.38%)
Ovary cancer cells: 22,333 (6.38%)
T cells: 20,433 (5.84%)
NK cells: 20,392 (5.83%)
Stem and progenitor cells: 19,759 (5.65%)
Endothelial cells: 19,759 (5.65%)
Epithelial cells: 19,759 (5.65%)
B cells: 19,339 (5.53%)
Myeloid cells: 17,892 (5.11%)
Lung cancer cells: 16,171 (4.62%)
Pericytes: 15,923 (4.55%)
Pancreas cancer cells: 14,416 (4.12%)
Stromal cells: 14,306 (4.09%)
Microglia: 7,290 (2.08%)
Astrocytes: 7,131 (2.04%)
Neurons: 5,134 (1.47%)
Liver cancer cells: 4,613 (1.32%)
